# Tratamento de Dados - Dengue 2022 (São Paulo)

Este notebook realiza o tratamento (saneamento) do arquivo `dengue2022_mes.xlsx`, que contém a distribuição de casos notificados e confirmados de dengue por município e mês em São Paulo em 2022. O processo segue diretrizes de análise descritiva baseadas no livro *Estatística Básica* (Morettin & Bussab, 6ª ed.) e no repositório https://github.com/Guilherme-Fernandes-da-Costa/RP2_2025, que explora dados epidemiológicos e saneamento.

## Objetivo
- Limpar e preparar os dados para análise.
- Calcular estatísticas descritivas e criar visualizações básicas.
- Preparar os dados para futuras correlações (ex.: com indicadores de saneamento).

In [ ]:
# Importação de bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 1. Leitura do Arquivo
O arquivo possui headers multi-nível (linhas 4-6) e linhas iniciais irrelevantes. Usamos `pd.read_excel` com `header=[3,4,5]` para capturar o multi-header.

In [ ]:
# Leitura do arquivo
df = pd.read_excel('dengue2022_mes.xlsx', sheet_name='dengue2022_mes', header=[3, 4, 5], skiprows=0)

# Visualizar as primeiras linhas e shape
print(df.head())
print(df.shape)

## 2. Limpeza Inicial (Saneamento Básico)
- Remover linhas vazias ou irrelevantes.
- Preencher NaNs com 0 em colunas numéricas.
- Renomear colunas para simplificar.
- Converter tipos para inteiro onde aplicável.

In [ ]:
# Remover linhas com todos NaNs e últimas linhas (totais e notas)
df = df.dropna(how='all')
df = df[:-3]  # Ajuste se necessário com base na análise dos dados

# Preencher NaNs em colunas numéricas com 0
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_cols] = df[numeric_cols].fillna(0)

# Flatten multi-header
df.columns = ['_'.join(col).strip().replace('Unnamed:', '').replace('_level_2', '') for col in df.columns.values]
df.columns = df.columns.str.replace(r'\s+', '_', regex=True)

# Converter tipos para inteiro
case_cols = [col for col in df.columns if 'notificados' in col or 'confirmados' in col]
df[case_cols] = df[case_cols].astype(int)

# Remover colunas desnecessárias
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

print(df.head())

## 3. Tratamento Avançado
- Calcular totais confirmados por mês (autóctones + importados).
- Adicionar total anual por município.
- Transformar em formato long para análise temporal.

In [ ]:
# Calcular totais confirmados por mês
months = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']
for month in months:
    df[f'{month}_confirmados_total'] = df[f'{month}_confirmados_autóctones'] + df[f'{month}_confirmados_importados']

# Total anual por município
df['Total_confirmados_anual'] = df[[f'{m}_confirmados_total' for m in months]].sum(axis=1)

# Filtrar municípios com >100 casos totais
df_filtered = df[df['Total_confirmados_anual'] > 100]

# Transformar em formato long
id_vars = ['DRS_', 'GVE_', 'Região_de_Saúde_', 'município_']
value_vars = [col for col in df.columns if 'confirmados_total' in col and 'anual' not in col]
df_long = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='Mês', value_name='Casos_Confirmados')
df_long['Mês'] = df_long['Mês'].str.replace('_confirmados_total', '')

print(df_long.head())

## 4. Análise Estatística Básica
- Estatísticas descritivas (média, mediana, desvio-padrão).
- Visualização com histograma (baseado no cap. 2 do livro).

In [ ]:
# Estatísticas descritivas por mês (ex.: Março)
print(df['Março_confirmados_total'].describe())

# Histograma de casos totais anuais
plt.figure(figsize=(10,6))
df['Total_confirmados_anual'].hist(bins=20)
plt.title('Distribuição de Casos Confirmados de Dengue por Município (2022)')
plt.xlabel('Casos Totais')
plt.ylabel('Frequência')
plt.show()

## 5. Exportação dos Dados
Salvar o dataframe limpo para uso posterior.

In [ ]:
# Exportar para CSV
df.to_csv('dengue2022_limpo.csv', index=False)
print('Arquivo salvo como dengue2022_limpo.csv')

## 6. Próximos Passos
- Integrar com dados de saneamento (ex.: IBGE/DataSUS) para análise de correlações.
- Expandir visualizações (boxplots, séries temporais).
- Adicionar análise regional por DRS/GVE.